In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv

from ML.DTGenerated import DTGenerated
import pickle

In [22]:
from generateTestingData.useGeneratedData import makeDataUsable, splitTrainTest, NewPredicter

In [23]:
df = makeDataUsable(read_csv("../data/testGenerateData.csv"),4, 3)
print(df.shape)
df.head()

(2369, 30)


,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,50.0,50.0,50.0,50.000,80.5604,100.0,100.0,100.0000,1.835,...,0.000,0.002,-0.000,-0.001,-0.001,0.007,0.006,0.001,0.000,0.0
1,4.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.661,...,-0.001,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,-0.001,0.0
2,5.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,4.117,...,-0.000,-0.001,-0.001,0.000,0.000,0.001,0.000,-0.001,0.000,0.0
3,6.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.865,...,0.000,-0.001,0.000,0.000,0.000,0.000,-0.001,0.000,0.000,0.0
4,7.0,50.0,50.0,50.0,5.322,100.0000,100.0,100.0,83.3296,3.704,...,0.004,0.000,0.000,0.000,-0.001,-0.001,0.000,0.000,0.005,0.0


In [24]:
df["COL"].value_counts()

0.0    2188
1.0     181
Name: COL, dtype: int64

In [25]:
df.corr()["COL"]

Time     -0.056514
TTC1     -0.102932
TTC2     -0.173143
TTC3     -0.218984
TTC4     -0.255912
DTO1     -0.067642
DTO2     -0.108841
DTO3     -0.155762
DTO4     -0.192370
JERK1    -0.025233
JERK2    -0.013158
JERK3     0.023716
JERK4     0.237693
Speed1    0.129512
Speed2    0.136681
Speed3    0.127932
Speed4   -0.002580
asX1     -0.018938
asX2     -0.004479
asX3     -0.017248
asX4     -0.000513
asY1      0.031292
asY2      0.007849
asY3     -0.002019
asY4     -0.026174
asZ1      0.033380
asZ2      0.027377
asZ3      0.045076
asZ4      0.008791
COL       1.000000
Name: COL, dtype: float64

In [ ]:

# sampleSize = int(len(data[data['COL'] == 1]) * undersampleRatio)
# print("Undersampling")
# data = concat([data[data["COL"] == 0].sample(sampleSize, random_state=1), data[data["COL"] == True]])

In [26]:
df = pd.concat([df[df["COL"] == False].sample(200, random_state=1), df[df["COL"] == True]])

In [27]:
data = df.copy().drop(columns=df.columns[0], axis=1)
trainX, trainY, testX, testY = splitTrainTest(data)

splitting at 304.


In [28]:
trainX.head(3)

,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,...,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4
226,4.181,3.303,2.389,1.448,52.4022,40.3287,28.5215,16.9475,0.451,0.012,...,0.000,-0.000,0.000,0.000,0.000,0.000,0.001,-0.000,-0.000,-0.000
64,50.000,50.000,50.000,50.000,0.6410,3.9126,13.3183,21.2978,1.463,1.209,...,-0.012,-0.025,0.238,0.719,0.856,-0.014,-0.004,0.012,0.008,-0.004
695,50.000,4.850,3.686,2.227,31.2345,25.6007,20.1456,12.9023,5.010,4.278,...,0.008,-0.009,-0.001,0.001,0.000,-0.000,0.003,0.002,0.009,-0.010


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [30]:
xgb_clf = NewPredicter(XGBClassifier(n_estimators=100, random_state=1))
xgb_trainX = xgb_clf.preProcess(trainX.copy())
xgb_testX = xgb_clf.preProcess(testX.copy())

xgb_trainY, xgb_testY = trainY.to_numpy(), testY.to_numpy()

Scaler is fitted


In [31]:
xgb_clf.fit(xgb_trainX, xgb_trainY)

In [32]:
len(xgb_testX[0])
xgb_pred = xgb_clf.predict(xgb_testX)
xgb_score = xgb_clf.getScore(xgb_testY, xgb_pred)

Total: 77, Collisions: 38
Accuracy: 0.7402597402597403, Precision: 0.75, Recall: 0.7105263157894737, F1: 0.7297297297297298
Confusion matrix:
[[30  9]
 [11 27]]


TODO - need more data, what is radar?

In [33]:
# model_score = f"{score[0][0]}-{score[0][1]}-{score[1][0]}-{score[1][1]}"
model_score = f"{xgb_score[0][0]}-{xgb_score[0][1]}-{xgb_score[1][0]}-{xgb_score[1][1]}"
model_score

'30-9-11-27'

In [41]:
xgb_clf.saveModel("xgb_gen", accuracy=model_score)

Model saved!
